In this notebook, we will train an Latend Dirichlet Allocation (LDA) model on tweets to learn a set of words which commonly appear together, hopefully corresponding to a topic.
We will apply the LDA training on the whole corpus of our tweets and extract 10 topics. Additionally, we will visualize the results using the pyLDAvis library.

Following, we will take these results to a different notebook for analysis. There, we will assign a topic distribution on each tweet given the words used in it and we will sum the topic distributions of all tweets corresponding to a state to conclude to the topic distribution per state.

In [2]:
from pymongo import MongoClient
import json

client = MongoClient()
db = client.Twitter

In [3]:
import pandas as pd
import time
import re
from nltk.tokenize import RegexpTokenizer
import HTMLParser # In Python 3.4+ import html 
import nltk
from nltk.corpus import stopwords

# LOAD data from Mongo

In [4]:
start_time = time.time()
#we are filtering out tweets of different languages and outside of the US
filter_query = { 
    "$and":[ {"place.country_code":"US"}, { "lang": "en" } ]
    }
#we are keeping only our fields of interest
columns_query = {
    'text':1,
    'entities.hashtags':1,
    'entities.user_mentions':1,
    'place.full_name':1,
    'place.bounding_box':1
}

tweets = pd.DataFrame(list(db.tweets.find(
    filter_query, 
    columns_query
    )#.limit()
                               )
                          )

elapsed_time = time.time() - start_time
print elapsed_time


16.0380530357


# Preproccessing

In [5]:
#parse state variable
tweets['state'] = map(lambda place_dict: place_dict['full_name'][-2:] ,tweets['place'])


In [6]:
tweets['state'].value_counts().head()

SA    130964
CA     57675
NY     37822
FL     34685
TX     27640
Name: state, dtype: int64

In [7]:
# #for one state only
# state = 'CA'
# tweets = tweets[tweets['state']==state]
len(tweets)

517724

In [8]:
def Clean(unescaped_tweet):
    '''This function takes a tweet as input and returns a tokenizing list.'''
    
    tokenizer = RegexpTokenizer(r'\w+')
    cleaned_tweet_tokens = tokenizer.tokenize(unescaped_tweet.lower())
    
    return cleaned_tweet_tokens


start_time = time.time() #Starts time

tweets['text'] = tweets['text'].apply(lambda tweet: re.sub(r"http\S+", "", tweet))

#########################################################

def trump_mention(tweet):
    trump_count = 0
    if ('trump' in tweet.lower()) or ('donald'  in tweet.lower()):
        
        return True
        
    return False

tweets['Trump'] = tweets['text'].apply(lambda tweet: trump_mention(tweet))

##############################################################

#tweet mentions --->@
#tweet hashtags --->#

#create two column with the the hashtags and the mentions
tweets['mentions'] = tweets['text'].apply(lambda tweet: re.findall(r'\@\w+',tweet))
tweets['hashtags'] = tweets['text'].apply(lambda tweet: re.findall(r'\#\w+',tweet))

#remove hashtags and mentions
tweets['text'] = tweets['text'].apply(lambda tweet: re.sub(r"\@\w+" , "", tweet))
tweets['text'] = tweets['text'].apply(lambda tweet: re.sub(r"\#\w+" , "", tweet))

#remove the numbers from the text
tweets['text'] =tweets['text'].apply(lambda tweet: ''.join([i for i in tweet if not i.isdigit()]))

trump_count = 0
clinton_count =0


#remove the names and surnames of the two candidates
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"Trump" , "", tweet))
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"Clinton" , "", tweet))
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"Donald" , "", tweet))
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"Hillary" , "", tweet))
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"USA" , "", tweet))
tweets['text'] =tweets['text'].apply(lambda tweet: re.sub(r"amp" , "", tweet))



#tokenize the text and add an extra column
tweets['token'] = tweets['text'].apply(lambda tweet: Clean(tweet))

tweets['token'] = tweets['token'].apply(lambda x: list(set(x)-set(stopwords.words('english'))))

elapsed_time = time.time() - start_time #time ends
print elapsed_time

tweets.head()

75.8121948242


,_id,entities,place,text,state,Trump,mentions,hashtags,token
0,59c10dfc66b388bfad9235af,"{u'user_mentions': [{u'id': 813286, u'indices'...","{u'bounding_box': {u'type': u'Polygon', u'coor...",\n\n \nALL IN COLLUSION TOGETHER \n\n \n\n \n...,LA,True,"[@BarackObama, @FBI, @LORETTALYNCH, @realDonal...","[#NOJUSTICE, #TrumpPence]","[collusion, together]"
1,59c10dfc66b388bfad9235b3,"{u'user_mentions': [], u'hashtags': [{u'indice...","{u'bounding_box': {u'type': u'Polygon', u'coor...","clear deliberately throwing this race,in h...",MD,True,[],"[#CNN, #newday, #Trump, #ISIS]","[mideast, destabilization, throwing, clear, kn..."
2,59c10dfc66b388bfad9235b4,"{u'user_mentions': [{u'id': 4852163069, u'indi...","{u'bounding_box': {u'type': u'Polygon', u'coor...",I TOTALLY CONCUR!! This Election is just CRA...,MD,True,"[@mike4193496, @realDonaldTrump]",[],"[blowing, last, mind, n, cra, election, concur..."
3,59c10dfc66b388bfad9235b5,"{u'user_mentions': [{u'id': 1339835893, u'indi...","{u'bounding_box': {u'type': u'Polygon', u'coor...","you ARE the co-founder of ISIS, you crooked, ...",TX,False,[@HillaryClinton],[],"[co, founder, crooked, evil, live, witch, isis..."
4,59c10dfc66b388bfad9235b6,"{u'user_mentions': [{u'id': 25073877, u'indice...","{u'bounding_box': {u'type': u'Polygon', u'coor...",", you wouldn't recognize a lie if it came from...",CA,True,[@realDonaldTrump],[#NeverTrump],"[lie, continually, mouth, came, recognize]"


In [9]:
tweets.head()

,_id,entities,place,text,state,Trump,mentions,hashtags,token
0,59c10dfc66b388bfad9235af,"{u'user_mentions': [{u'id': 813286, u'indices'...","{u'bounding_box': {u'type': u'Polygon', u'coor...",\n\n \nALL IN COLLUSION TOGETHER \n\n \n\n \n...,LA,True,"[@BarackObama, @FBI, @LORETTALYNCH, @realDonal...","[#NOJUSTICE, #TrumpPence]","[collusion, together]"
1,59c10dfc66b388bfad9235b3,"{u'user_mentions': [], u'hashtags': [{u'indice...","{u'bounding_box': {u'type': u'Polygon', u'coor...","clear deliberately throwing this race,in h...",MD,True,[],"[#CNN, #newday, #Trump, #ISIS]","[mideast, destabilization, throwing, clear, kn..."
2,59c10dfc66b388bfad9235b4,"{u'user_mentions': [{u'id': 4852163069, u'indi...","{u'bounding_box': {u'type': u'Polygon', u'coor...",I TOTALLY CONCUR!! This Election is just CRA...,MD,True,"[@mike4193496, @realDonaldTrump]",[],"[blowing, last, mind, n, cra, election, concur..."
3,59c10dfc66b388bfad9235b5,"{u'user_mentions': [{u'id': 1339835893, u'indi...","{u'bounding_box': {u'type': u'Polygon', u'coor...","you ARE the co-founder of ISIS, you crooked, ...",TX,False,[@HillaryClinton],[],"[co, founder, crooked, evil, live, witch, isis..."
4,59c10dfc66b388bfad9235b6,"{u'user_mentions': [{u'id': 25073877, u'indice...","{u'bounding_box': {u'type': u'Polygon', u'coor...",", you wouldn't recognize a lie if it came from...",CA,True,[@realDonaldTrump],[#NeverTrump],"[lie, continually, mouth, came, recognize]"


In [10]:
#test['tags'] = map(lambda tweet: map(lambda tweet: tweet['text'] , tweet['entities']['hashtags']) if tweet['entities']['hashtags'] != None else None, raw_tweet[:100])
#tweets['text'][9]

In [11]:
doc_complete = tweets['token'].tolist()
doc_complete[:2]

[[u'collusion', u'together'],
 [u'mideast',
  u'destabilization',
  u'throwing',
  u'clear',
  u'knew',
  u'started',
  u'race',
  u'deliberately',
  u'w',
  u'invasion',
  u'iraq']]

In [12]:
import gensim

In [13]:
import pickle

In [14]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(doc_complete)
pickle.dump(dictionary, open( 'dictionary2.pickle', "wb" ) )

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_complete]
pickle.dump(doc_term_matrix, open( 'doc_term_matrix.pickle', "wb" ) )


In [15]:
Lda = gensim.models.ldamulticore.LdaMulticore

In [ ]:
nr_topics = 10
nr_passes = 100

start_time = time.time()
# Creating the object for LDA model using gensim library

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=nr_topics, id2word = dictionary, passes=nr_passes)

elapsed_time = time.time() - start_time


In [18]:
print 'Topic modelling for', nr_topics,'topics,', nr_passes,'passes,',len(tweets),'tweets:','\ncomplete in',elapsed_time/60.,'minutes'

Topic modelling for 10 topics, 100 passes, 517724 tweets: 
complete in 88.7028 minutes


In [ ]:
# Runtimes:
# Florida (~4K) ~ 16 min on 10 topics, 300 passes
# CA (57K) - 48 min on 10 topics 300 passes


In [ ]:
# can we do it on the whole data -> take the topics and classify each tweet within them.
# then we have discrete sets with topics and words weights in each topic.
# so then isn't a tweet represented by the appropriate values?

In [ ]:
# Print 2 topics and describe then with 4 words.
topics = ldamodel.print_topics(num_topics=nr_topics, num_words=50)
i=0
for topic in topics:
    print topic
    print ""
    i+=1

# save/load the model

In [5]:
import pickle

In [7]:
nr_topics = 10
nr_passes = 100


In [8]:
state = 'allstates'
name = "trained models/lda/%s_%itopics_%ipasses.pickle"%(state,nr_topics,nr_passes)
print "Procceed to save model in:", name

Procceed to save model in: trained models/lda/allstates_10topics_100passes.pickle


In [ ]:
pickle.dump(ldamodel, open( name, "wb" ) )

In [9]:
#load
ldamodel = pickle.load(open(name,'rb'))

/home/antonis/anaconda2/envs/USelections/lib/python2.7/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/home/antonis/anaconda2/envs/USelections/lib/python2.7/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# efforts with pyLDAvis (visualize the LDA topics)

In [13]:
import time

In [10]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [14]:
#load the LDA results (model, dictionary and corpus)
start_time = time.time()

ldamodel = pickle.load(open('trained models/lda/allstates_10topics_100passes.pickle'))
dictandcorpus = pickle.load(open('trained models/lda/Dictionary.pickle'))
c = dictandcorpus[1]
d = dictandcorpus[0]
del dictandcorpus

elapsed_time = time.time() - start_time
print elapsed_time

19.5377728939


In [18]:
data = pyLDAvis.gensim.prepare(ldamodel, c, d)

/home/antonis/anaconda2/envs/USelections/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [19]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      15.327588        1       1  0.139161  0.097866
4      13.260427        1       2  0.118236  0.055774
0      11.438490        1       3  0.091175  0.034620
7      10.038032        1       4  0.041942 -0.096417
1       9.475785        1       5  0.063025  0.126108
8       9.437777        1       6  0.009806 -0.257452
5       9.389313        1       7  0.059145 -0.007431
6       8.798108        1       8  0.009951  0.183752
3       7.918884        1       9 -0.061873 -0.221489
9       4.915595        1      10 -0.470568  0.084669, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
16686  Default  13497.000000          today  13497.000000  30.0000  30.0000
11065  Default  18750.000000              u  18750.000000  29.0000  29.0000
20701  Default  14314.000000           know  14314.000000  28.0000  28.0000
49702  Default   8713.000000              f   8713.000000  27.0000  27.0000
31519  Default  10536.000000      president  10536.000000  26.0000  26.0000
4277   Default   7556.000000       pressure   7556.000000  25.0000  25.0000
18205  Default   7380.000000           wind   7380.000000  24.0000  24.0000
39720  Default   7310.000000           temp   7310.000000  23.0000  23.0000
39346  Default   7283.000000           rain   7283.000000  22.0000  22.0000
30116  Default   7267.000000             ky   7267.000000  21.0000  21.0000
24872  Default   7258.000000            mph   7258.000000  20.0000  20.0000
67159  Default   7247.000000       forecast   7247.000000  19.0000  19.0000
44884  Default   7235.000000           crab   7235.000000  18.0000  18.0000
7652   Default   7232.000000        orchard   7232.000000  17.0000  17.0000
22490  Default   7232.000000            hpa   7232.000000  16.0000  16.0000
7798   Default  10682.000000             go  10682.000000  15.0000  15.0000
56499  Default  10469.000000          right  10469.000000  14.0000  14.0000
53501  Default  17342.000000            get  17342.000000  13.0000  13.0000
35473  Default   6793.000000           fine   6793.000000  12.0000  12.0000
6116   Default  12084.000000        america  12084.000000  11.0000  11.0000
27280  Default   9926.000000          never   9926.000000  10.0000  10.0000
46696  Default   9623.000000            say   9623.000000   9.0000   9.0000
13096  Default  23346.000000           like  23346.000000   8.0000   8.0000
33563  Default   7017.000000              r   7017.000000   7.0000   7.0000
18853  Default  18184.000000         people  18184.000000   6.0000   6.0000
21941  Default   8424.000000           said   8424.000000   5.0000   5.0000
44341  Default   6288.000000           look   6288.000000   4.0000   4.0000
46731  Default   9454.000000           make   9454.000000   3.0000   3.0000
19212  Default   6469.000000           love   6469.000000   2.0000   2.0000
27513  Default   6518.000000      americans   6518.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
4277   Topic10   7555.203455       pressure   7556.081356   3.0126  -3.0861
18205  Topic10   7379.913977           wind   7380.791875   3.0126  -3.1096
39720  Topic10   7309.322080           temp   7310.199978   3.0126  -3.1192
39346  Topic10   7282.801213           rain   7283.679111   3.0126  -3.1229
30116  Topic10   7266.239658             ky   7267.117556   3.0126  -3.1251
24872  Topic10   7257.209307            mph   7258.087204   3.0126  -3.1264
67159  Topic10   7246.434932       forecast   7247.312829   3.0126  -3.1279
44884  Topic10   7235.079351           crab   7235.957248   3.0126  -3.1294
7652   Topic10   7231.958671        orchard   7232.836567   3.0126  -3.1299
22490  Topic10   7231.958671            hpa   7232.836567   3.0126  -3.1299
35473  Topic10   6792.753869           fi

# Interactive version available [here](https://codepen.io/Martin13131/full/zEwxxx/)
## Also available as an html document in this git repository (LDA_topics.html)


In [ ]:
#save results as an html file
pyLDAvis.save_html(data, open('LDA topics.html','wb'))